In [87]:
import os
import re
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [59]:
batch_size = 16

In [60]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator      = datagen.flow_from_directory('data/train',
                                                    target_size=(32, 32),
                                                    batch_size=batch_size,
                                                    subset='training')

validation_generator = datagen.flow_from_directory('data/train',
                                                    target_size=(32, 32),
                                                    batch_size=batch_size,
                                                    subset='validation')

Found 40000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [61]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [78]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              steps_per_epoch=40000 // batch_size,
                              validation_steps=10000 // batch_size)

Epoch 1/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.6181 - accuracy: 0.7707 - val_loss: 2.1699 - val_accuracy: 0.6790
Epoch 2/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.6130 - accuracy: 0.7717 - val_loss: 1.4046 - val_accuracy: 0.6827
Epoch 3/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.6025 - accuracy: 0.7748 - val_loss: 0.8994 - val_accuracy: 0.6845
Epoch 4/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.5975 - accuracy: 0.7741 - val_loss: 1.6083 - val_accuracy: 0.6843
Epoch 5/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.5880 - accuracy: 0.7810 - val_loss: 1.9512 - val_accuracy: 0.6834
Epoch 6/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.5884 - accuracy: 0.7792 - val_loss: 0.7037 - val_accuracy: 0.6876
Epoch 7/10
2500/2500 [==============================] - 16s 6ms/step - loss: 0.5839 - accuracy: 0.7783 - val_loss: 1.4227 - val_ac

In [77]:
model.save_weights('69_640s_batch16_epoch10_cnv32_cnv64_dns64_drop0.5_dns10_adam.h5')

In [80]:
model.load_weights('69_480s_batch16_epoch10_cnv32_cnv64_dns64_drop0.5_dns10_adam.h5')

In [81]:
from IPython.display import SVG
from keras.utils import model_to_dot
SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [83]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('data/test',
                                                   target_size=(32, 32),
                                                   batch_size=batch_size, 
                                                   shuffle=False)

Found 300000 images belonging to 1 classes.


In [84]:
pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)

18750/18750 [==============================] - 266s 14ms/step


In [85]:
indices = np.argmax(pred, axis=1)

In [88]:
ids = [int(re.sub('\D+', '', name)) for name in test_generator.filenames]

In [89]:
flipped = {v:k for k,v in validation_generator.class_indices.items()}

In [90]:
labels = [flipped[i] for i in indices]

In [91]:
submission = pd.DataFrame({'id':ids, 'label':labels}).sort_values(['id']).reset_index(drop=True)

In [92]:
submission.to_csv('submission.csv', index=False)